> **The nostr community is awesome and there is no shortage of other resources - make sure you check them out:**
>  - [python-nostr](https://github.com/jeffthibault/python-nostr) - the original repository that I forked to make this tutorial
>  - [awesome-nostr](https://github.com/aljazceru/awesome-nostr) -  a curated github repository of great resources
>  - [nostr](https://github.com/nostr-protocol/nostr) - the github for the nostr protocol
>   - including my favorite - [Nostr Improvement Proposals (NIPs)](https://github.com/nostr-protocol/nips) - the list of implemented proposals that clearly describe usage of the nostr protocol

# getting started

If you have [conda](https://docs.conda.io/en/latest/) or [mamba](https://mamba.readthedocs.io/en/latest/) installed you can use the cell below to create an environment that can be used to run this jupyter notebook in jupyter or vscode
```python
mamba env create --file './nostr.yml'
mamba activate nostr
```
or
```python
conda env create --file './nostr.yml'
conda activate nostr
```

## make sure this notebook can access the python-nostr code
run the cell below to add the parent directory to your python path and make sure `python-nostr` is in your path

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path('../').resolve()))
assert 'python-nostr' in [Path(path).name for path in sys.path]


## key management 

> description of keys in nostr can be found in ([NIP-01](https://github.com/nostr-protocol/nips/blob/master/01.md) and [NIP-19](https://github.com/nostr-protocol/nips/blob/master/19.md))

python-nostr has basic key management capabilities to generate new private keys or load existing keys.
The base key class is instantiated with raw_bytes - if you are loading an existing hex key you can use `Key.from_hex()`

The `PublicKey` and `PrivateKey` classes inherit these capabilities. The bech32 representation of the private and public key (i.e. npub... and nsec...) are defined in each class respectively.

**change log from original repo:**
 - Added base Key class to manage shared methods
 - Added hex to key method
 - Changed `raw_secret` to `raw_bytes` in `PrivateKey` to keep classes consistent
    - this change helps the code, but does it add some risk to users?

In [2]:
from pathlib import Path
import secrets
from nostr.key import Key, PrivateKey, PublicKey

base_key = Key(secrets.token_bytes(32))
print(f'this is the raw bytes of a key: {base_key.raw_bytes}\n'
      f'this is the hex of the same key: {base_key.hex()}')

assert base_key.hex() == Key.from_hex(base_key.hex()).hex()
assert issubclass(PrivateKey, Key) and issubclass(PublicKey, Key)

try:
      base_key.bech32()
except ValueError as e:
      print(e)

this is the raw bytes of a key: b'\x11E\x8d\xe7%\xb3^\xde\xcfG\xee\x1e\x04x\x90\xa1\x9a\xc4s\xab"\x1aX\xef\xe6%\xb5B\x04\xe0\x95\n'
this is the hex of the same key: 11458de725b35edecf47ee1e047890a19ac473ab221a58efe625b54204e0950a
bech32 prefix not defined for type <class 'nostr.key.Key'>


Below we use the `PrivateKey` class to make a private key or load it from a file on disk. We can also confirm that the private and public key representations start with the appropriate characters (npub and nsec)

In [3]:
key_file = Path('./private_key.txt') # saving a private key for testing

if not key_file.exists():
    private_key = PrivateKey()
    public_key = private_key.public_key
    print(f"Generated new private key")
    print(f"Public key: {public_key.bech32()}")
    with open(key_file, 'w') as f:
        f.write(private_key.raw_bytes.hex())
else:
    with open(key_file, 'r') as f:
        hex = f.read()
        # private_key = PrivateKey(bytes.fromhex(hex))
        private_key = PrivateKey.from_hex(hex)
    public_key = private_key.public_key
    print(f'Loaded private key from {key_file}')
    print(f'Public key: {public_key.bech32()}')

assert private_key.bech32()[:4]=='nsec' and public_key.bech32()[:4]=='npub'

Loaded private key from private_key.txt
Public key: npub19nqc7plddvevd8sgfpahzuput0q9hwrfhk8crj9k73fpg8qwjx3sk6224n


## relay manangement

> a basic description of relays in nostr can be found in [NIP-01](https://github.com/nostr-protocol/nips/blob/master/01.md)

Next let's instantiate a `RelayManager` and add a couple relay urls. We are going to set one to not write and see what happens.

We will also attach a `message_store` dict to our relay manager where we can store messages when we get around to requesting them - I am making this now, because the `RelayManager` class automatically creates a `MessagePool`, which will control incoming messages, but it will not save them.

**changelog from original repo:**
 - added `__iter__` method to the `RelayManager` class

In [4]:
from nostr.relay_manager import RelayManager, Relay

relay_manager = RelayManager()
relay_manager.add_relay('wss://nostr-2.zebedee.cloud')
relay_manager.add_relay('wss://nostr.zebedee.cloud')
relay_manager.add_relay('wss://nostr-relay.lnmarkets.com', write=False)

relay_manager.message_store = {}

the relay manager keeps all of the `Relay` objects in a dict keyed by the url. Let's look at some of the properties of the relays we added before we actually do anything

**changelog from original repo:**
 - added `__repr__` of json object for `Relay`

In [5]:
for relay in relay_manager:
    print(relay, '\n\n')
    assert type(relay) == Relay


{
  "url": "wss://nostr-2.zebedee.cloud",
  "policy": {
    "read": true,
    "write": true
  },
  "subscriptions": []
} 


{
  "url": "wss://nostr.zebedee.cloud",
  "policy": {
    "read": true,
    "write": true
  },
  "subscriptions": []
} 


{
  "url": "wss://nostr-relay.lnmarkets.com",
  "policy": {
    "read": true,
    "write": false
  },
  "subscriptions": []
} 




Now we can connect - in this case we will connect and test that all relays have a websocket (just a naive test that they are not None when open and are None when closed).

**changelog from original repo:**
 - added a connection context for easier operations
    - is this actually helpful in context of nostr?


In [6]:
import ssl
from nostr.relay_manager import Connection

with relay_manager.connection({"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
    for relay in relay_manager:
        assert relay.ws.sock is not None

for relay in relay_manager:
    assert relay.ws.sock is None

## writing to the relays

> a basic description of events that can be written to a relay can be found in [NIP](https://github.com/nostr-protocol/nips/blob/master/01.md). Later NIPs include more detailed information about various kinds of events and associated metadata.

Here we will write a very simple kind 1 (a note) event to a relay. We are not including any additional information in the event like tags. I hope to add more detail about those into this tutorial at a later date.

In [7]:
import json 
import ssl
import time
from nostr.event import Event
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType
from nostr.key import PrivateKey

text = 'Hello Nostr - testing from python-nostr'
event = Event(private_key.public_key.hex(), text)
event.sign(private_key.hex())

message = json.dumps([ClientMessageType.EVENT, event.to_json_object()])
print(message)

with relay_manager.connection({'cert_reqs': ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
    time.sleep(2)
    relay_manager.publish_message(message)
    time.sleep(1) # allow the messages to send

["EVENT", {"id": "38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105", "pubkey": "2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3", "created_at": 1672011680, "kind": 1, "tags": [], "content": "Hello Nostr - testing from python-nostr", "sig": "eae20a19a377b875c147ec6668c83ed3ddc5eabec7b1c3fc2714f6de9028e82bc499f31d29ab8f5ae77bc8db8160ee08e952bb4d3425e8b5cbaad12a4693fd91"}]


## requesting from the relays
we can use python-nostr to make requests to send to the relays. First we will try to request the event id we just published.

> **_NOTE:_** when an event is processed by python noster it's ID is stored in `RelayManager.message_pool._unique_events`. This ID won't get processed a second time. If you are just exploring in a notebook like this you will need to clear this attribue to process the event a second time.

In [8]:
import json
import ssl
import time
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType


filters = Filters([Filter(ids=[event.id])])
subscription_id = 'single_event'
request = [ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())
message = json.dumps(request)

# checking what we made above
print(f'the request is: {request}')
print(f'the message to the relay is: {message}')

relay_manager.add_subscription(subscription_id, filters)
print('The following subscriptions have been added to relays:')
for relay in relay_manager:
    print(f'{relay.url}:', relay.subscriptions.get(subscription_id)
                                .to_json_object())

the request is: ['REQ', 'single_event', {'ids': ['38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105']}]
the message to the relay is: ["REQ", "single_event", {"ids": ["38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105"]}]
The following subscriptions have been added to relays:
wss://nostr-2.zebedee.cloud: {'id': 'single_event', 'filters': [{'ids': ['38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105']}]}
wss://nostr.zebedee.cloud: {'id': 'single_event', 'filters': [{'ids': ['38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105']}]}
wss://nostr-relay.lnmarkets.com: {'id': 'single_event', 'filters': [{'ids': ['38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105']}]}


In [9]:
import pprint
import warnings

with Connection(relay_manager, {'cert_reqs': ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
  previous_event_ids = len(relay_manager.message_pool._unique_events)
  print(f'{previous_event_ids} events already received\n')
  if event.id in relay_manager.message_pool._unique_events:
    warnings.warn(f'event {event.id} has already been processed and won\'t display below. '
                   'Please reinitiate the message pool to receive it again')
                   
  time.sleep(1.25) # allow the connections to open
  relay_manager.publish_message(message)
  time.sleep(1) # allow the messages to send

  while relay_manager.message_pool.has_events():
    event_msg = relay_manager.message_pool.get_event()
    relay_manager.message_store.update({event_msg.event.id: event_msg})
    print(pprint.pprint(event_msg.event.to_json_object(), indent=2))
  while relay_manager.message_pool.has_notices():
    notice_msg = relay_manager.message_pool.get_notice()
    print(pprint.pprint(notice_msg.event.to_json_object(), indent=2))
  while relay_manager.message_pool.has_eose_notices():
    eose_notice_msg = relay_manager.message_pool.get_eose_notice()
    print(f'end of {eose_notice_msg.subscription_id} received.')

0 events already received

{ 'content': 'Hello Nostr - testing from python-nostr',
  'created_at': 1672011680,
  'id': '38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105',
  'kind': 1,
  'pubkey': '2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3',
  'sig': 'eae20a19a377b875c147ec6668c83ed3ddc5eabec7b1c3fc2714f6de9028e82bc499f31d29ab8f5ae77bc8db8160ee08e952bb4d3425e8b5cbaad12a4693fd91',
  'tags': []}
None
end of single_event received.


Now let's try to request the last 10 events from a user

**TODO:** add other event types and examples below

In [10]:
import json
import ssl
import time
from nostr.filter import Filter, Filters
from nostr.event import Event, EventKind
from nostr.relay_manager import RelayManager
from nostr.message_type import ClientMessageType


filters = Filters([Filter(authors=['82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2'],
                          kinds=[EventKind.TEXT_NOTE],
                          limit=5
                          )])
subscription_id = 'last10_jack'
request = [ClientMessageType.REQUEST, subscription_id]
request.extend(filters.to_json_array())
message = json.dumps(request)

# checking what we made above
print(f'the request is: {request}')
print(f'the message to the relay is: {message}')

relay_manager.add_subscription(subscription_id, filters)
print('The following subscriptions have been added to relays:')
for relay in relay_manager:
    print(f'{relay.url}:')
    pprint.pprint(relay.subscriptions.get(subscription_id)
                       .to_json_object(), indent=2)
    print('\n')

the request is: ['REQ', 'last10_jack', {'kinds': [<EventKind.TEXT_NOTE: 1>], 'authors': ['82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2'], 'limit': 5}]
the message to the relay is: ["REQ", "last10_jack", {"kinds": [1], "authors": ["82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2"], "limit": 5}]
The following subscriptions have been added to relays:
wss://nostr-2.zebedee.cloud:
{ 'filters': [ { 'authors': [ '82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2'],
                 'kinds': [<EventKind.TEXT_NOTE: 1>],
                 'limit': 5}],
  'id': 'last10_jack'}


wss://nostr.zebedee.cloud:
{ 'filters': [ { 'authors': [ '82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2'],
                 'kinds': [<EventKind.TEXT_NOTE: 1>],
                 'limit': 5}],
  'id': 'last10_jack'}


wss://nostr-relay.lnmarkets.com:
{ 'filters': [ { 'authors': [ '82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2'],
       

In [11]:
import warnings

messages = {}

with Connection(relay_manager, {"cert_reqs": ssl.CERT_NONE}): # NOTE: This disables ssl certificate verification
  previous_event_ids = len(relay_manager.message_pool._unique_events)
  print(f'{previous_event_ids} events already received\n')

  time.sleep(1.25) # allow the connections to open
  relay_manager.publish_message(message)
  time.sleep(1) # allow the messages to send
  
  while relay_manager.message_pool.has_events():
    event_msg = relay_manager.message_pool.get_event()
    relay_manager.message_store.update({event_msg.event.id: event_msg})
    print(pprint.pprint(event_msg.event.to_json_object(), indent=2))
  while relay_manager.message_pool.has_notices():
    notice_msg = relay_manager.message_pool.get_notice()
    print(pprint.pprint(notice_msg.event.to_json_object(), indent=2))
  while relay_manager.message_pool.has_eose_notices():
    eose_notice_msg = relay_manager.message_pool.get_eose_notice()
    print(f'end of {eose_notice_msg.subscription_id} received.')

1 events already received

{ 'content': 'No. I’m pretty quiet',
  'created_at': 1672010344,
  'id': 'f83d1d06cf29d43805a18923b9a16d7e579eb3b058027ca8256d98a69736eb46',
  'kind': 1,
  'pubkey': '82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2',
  'sig': '1fa7d54420da5d37987bd65865be0f89112bd3236f07d2ec773390e314b43177971f9e8bf3292d18b0b655bd4df22adbec73f253467cfd37548bf94f925ac2f7',
  'tags': [ [ 'e',
              '684d45b8a395b51bc38f66863c752bfa9c40fe0451e023c66e63b497a292f08a',
              'wss://nostr-pub.wellorder.net'],
            [ 'e',
              'fe46af93ea3fe98eef1f42d5b7c284ec2fd813a64574ad5fbb195512ff873f2c'],
            [ 'p',
              '5aec57f756b06f92dded240bb0122771bbe5d57e444ac243da4d708f807528d0',
              'wss://nostr-pub.wellorder.net'],
            [ 'p',
              'c1deeda422c79b645ee58d8cb1b839a21a25926c9d94c21628ce8e2d89c3dbc8',
              'wss://nostr.rocks'],
            [ 'p',
              'c134bfc9082e1337ce67013fe0f

## checking out the received events

remember we saved our messages into a dict we attached to the `RelayManager`. Typically, a client will be saving messages in a more robust way, but this was good enough to see how the `python-nostr` functions.

In [12]:
pprint.pprint(relay_manager.message_store, indent=2)

{ '10900f56e69ee0a7036fd21328d8eb2d54430cf846da42abb499525af7825b6b': <nostr.message_pool.EventMessage object at 0x10baf2770>,
  '322d0649a93b0a17eac005e85fef4f847536da75f54a4269b64d53e7afc9b028': <nostr.message_pool.EventMessage object at 0x10baf1750>,
  '38cf9214618636479f91c10b55fc3a8c2501dc89e0451d3c6d32475992f81105': <nostr.message_pool.EventMessage object at 0x1096bc910>,
  '9730a1f621be24d70a724a72ebe1157abaddbfc8d7be015cc95f6f28e7554164': <nostr.message_pool.EventMessage object at 0x10baf17e0>,
  'a036836d4280831bc69464f957536a1e5652fcddd695df77ba4514d9db610542': <nostr.message_pool.EventMessage object at 0x10baf2890>,
  'f83d1d06cf29d43805a18923b9a16d7e579eb3b058027ca8256d98a69736eb46': <nostr.message_pool.EventMessage object at 0x10baf3dc0>}


In [15]:
for event_id, event_msg in relay_manager.message_store.items():
    print(f'author: {event_msg.event.public_key}')
    print(f'\t{event_msg.event.content}\n')

author: 2cc18f07ed6b32c69e08487b71703c5bc05bb869bd8f81c8b6f452141c0e91a3
	Hello Nostr - testing from python-nostr

author: 82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2
	No. I’m pretty quiet

author: 82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2
	It’s the future of veganism #[7]

author: 82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2
	I was vegan for 2 years. A friend for over 25. Worst health for both of us.

author: 82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2
	Never heard of it

author: 82341f882b6eabcd2ba7f1ef90aad961cf074af15b9ef44a09f9d2a8fbfbe6a2
	ftw

